## Import the necessary libraries:

In [3]:
import io
!pip install PyPDF2
import PyPDF2
import chardet
import re
import pandas as pd

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 15.2 MB/s eta 0:00:00


# Load the dataset

In [4]:
skills_data = pd.read_csv('/content/skills.csv')

## Function to parse txt file

In [13]:
def parse_resume(file_path):
    text = ""

    # Open the file
    with open(file_path, 'rb') as file:
        raw_data = file.read()
        encoding_result = chardet.detect(raw_data)
        encoding = encoding_result['encoding']
    with open(file_path, 'r', encoding=encoding) as file:
        resume_text = file.read()

    # Extract name, email, and phone number
    name_pattern = r"([A-Z][a-z]+)\s+([A-Z][a-z]+)"
    name_match = re.search(name_pattern, resume_text)
    name = name_match.group() if name_match else None

    email_pattern = r"\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b"
    email_match = re.search(email_pattern, resume_text, re.I)
    email = email_match.group() if email_match else None

    phone_pattern = r"\(?\d{3}\)?[-.\s]?\d{3}[-.\s]?\d{4}"
    phone_match = re.search(phone_pattern, resume_text)
    phone = phone_match.group() if phone_match else None

    # Extract skills
    skills_datasets = [skills_data]
    skills = []
    for dataset in skills_datasets:
        skills_pattern = r"\b(?:{})\b".format("|".join(map(re.escape, dataset['Skill'].dropna())))
        skills_match = re.findall(skills_pattern, resume_text, re.IGNORECASE)
        skills.extend(skills_match)

    skills = ', '.join(skills) if skills else None

    # Extract experience
    experience_pattern = r"EXPERIENCE\n+(.*?)(?:\n{2,}|$)"
    experience_match = re.search(experience_pattern, resume_text, re.DOTALL | re.IGNORECASE)
    experience = experience_match.group(1).strip() if experience_match else None

    experience = re.sub(r'\n\s*', ' ', experience) if experience else None

    # Extract education
    education_pattern = r"EDUCATION\n+(.*?)(?:\n{2,}|$)"
    education_match = re.search(education_pattern, resume_text, re.DOTALL | re.IGNORECASE)
    education = education_match.group(1).strip() if education_match else None

    # Return the extracted data as a dictionary
    extracted_data = {
        'name': name,
        'email': email,
        'phone': phone,
        'skills': skills,
        'experience': experience,
        'education': education
    }
    return extracted_data

## Testing

In [15]:
file = '/content/resume.txt'
data = parse_resume(file)
print("Name:", data.get('name'))
print("Email:", data.get('email'))
print("Phone:", data.get('phone'))
print("Skills:", data.get('skills'))
print("Experience:", data.get('experience'))
print("Education:", data.get('education'))

Name: Casey Hall
Email: casey.z.hall@gmail.com
Phone: 916-566-6124
Skills: gmail, .com
, analytical, customer service, banking, BANKING, due diligence, international, operations, risk assessments, banking, reports, marketing, sales, payments, Customer Service, banking, customer service, Economics
, finance, pressure
, Financial planning
, Risk assessment, Customer service, Problem solving
, Technical skills, CFA
Experience: BANKING ANALYST RJ Savings and Trust Dec 2016- Dec 2019 Managed 100 investors’ due diligence processes and coordinated with international operations managers when required. Performed financial risk assessments and studied banking and financial market trends to prepare activity and progress reports to management. Navigated marketing activities by reaching out to 150+ VC and lead 5% prospects. Key achievement Conducted valuation report on $20 million swaps deal for a tech company which resulted in 35% rise in corporate earnings. ASSOCIATE BANKER Merriweather Bank Sept